In [1]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationSummaryMemory
from langchain_community.llms.ctransformers import CTransformers
from langchain.prompts import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains.api.base import LLMChain
from langchain_community.chat_message_histories import ChatMessageHistory
from operator import itemgetter

In [2]:
llm = CTransformers(model="../tinyllama-1.1b-chat-v1.0.Q5_K_M.gguf", model_type="Llama", config={
    "temperature": 0,
    "stop": ['User', '\nUser','\n\n', '\nHuman', "</s>", " Human:"]
})

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
summary_prompt = PromptTemplate.from_template(
    (
        "<|system|>Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n"
        "Current summary:\n"
        "{summary}</s>\n\n"
        "<|user|>\nNew lines of conversation:\n"
        "{new_lines}</s>\n\n"
        "New summary:\n"
    )
)

In [12]:
chat_history = ChatMessageHistory()
# memory = ConversationBufferMemory(memory_key="history", chat_memory=chat_history, k=3)
# memory = ConversationSummaryBufferMemory(llm=llm, chat_memory=chat_history,) #max_token_limit=256, prompt=summary_prompt)
memory = ConversationSummaryMemory(chat_memory=chat_history, llm=llm, prompt=summary_prompt)

In [13]:
prompt = PromptTemplate.from_template(
    (
        "This is a conversation between human and AI. Reply to the user in NO MORE THAN 30 words.\n"
        "History: {history}</s>"
        "{input}</s>\n"
        "AI:"
    )
)

In [14]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
)

In [15]:
chain.invoke(input={
    "input": "Hello",
})

{'input': 'Hello', 'history': '', 'text': ' Hello, how are you?'}

In [16]:
chain.invoke(input={
    "input": "I'm fine. Thanks for asking.",
})

{'input': "I'm fine. Thanks for asking.",
 'history': "Human: Hi, how are you doing today? AI: I'm doing great! How about you?",
 'text': ' So, what do you want to talk about today?'}

In [17]:
chain.invoke(
    {
        "input": "I was reading an interesting book. It is finished now. Is there any books you would like to recommend?"
    }
)

{'input': 'I was reading an interesting book. It is finished now. Is there any books you would like to recommend?',
 'history': "Human: I'm just happy to hear that you're doing well. AI: That's great to hear! What are you up to today?",
 'text': ' Sure, here are some books I think you might enjoy: "The Alchemist" by Paulo Coelho, "The Hitchhiker\'s Guide to the Galaxy" by Douglas Adams, and "The Catcher in the Rye" by J.D. Salinger.'}

In [18]:
chain.invoke({
    "input": "Tell me more about the second one."
})

{'input': 'Tell me more about the second one.',
 'history': 'Human: That sounds interesting! Can you recommend a book that I might enjoy? AI: Sure thing! "The Alchemist" is a great choice for someone who loves fantasy and adventure. "The Hitchhiker\'s Guide to the Galaxy" is a funny and thought-provoking read, while "The Catcher in the Rye" is a classic mystery novel that will keep you guessing until the end.',
 'text': ' Sure thing! "The Catcher in the Rye" is a classic mystery novel by Agatha Christie. It\'s set in a small village and features a group of amateur detectives who solve a murder case. The book is known for its clever plot twists and witty dialogue, making it a great read for fans of mystery novels.'}

In [19]:
chain.invoke({
    "input": "No. Not that book. I ask about the \"Guide to the Galaxy\"."
})

{'input': 'No. Not that book. I ask about the "Guide to the Galaxy".',
 'history': 'Human: That sounds interesting! Can you recommend a book that I might enjoy? AI: Sure thing! "The Catcher in the Rye" is a classic mystery novel by Agatha Christie. It\'s set in a small village and features a group of amateur detectives who solve a murder case. The book is known for its clever plot twists and witty dialogue, making it a great read for fans of mystery novel.',
 'text': " Oh, yes! That's a classic science fiction novel by Douglas Adams. It follows the adventures of a group of space travelers as they explore the universe and encounter alien life forms. The book is known for its humor, wit, and exploration of humanity's place in the cosmos. No problem there. I'll take that one!"}